<center>
<img src="https://wherenext.esri.de/wp-content/uploads/2019/02/Logo_CH_DE_Esri-800x130.png"/>
<h1> EsriKon 2020: Spatial Data Science - Datenaufbereitung mit ArcGIS Notebooks</h1>
<br><br>

# EnviroCar - Daten sammeln & analysieren


### Mit GIS verbinden & Libraries laden:

In [ ]:
from arcgis.gis import GIS
gis = GIS("home")

In [ ]:
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis import env
import requests

### Daten mit Open Data API abfragen

In [ ]:
tracks_overview = requests.get("https://envirocar.org/api/stable/tracks").json()
tracks_overview

### Pandas

<center>
<img src="https://demoportal.esri.de/bilder/esrikon2020/pandas.gif"/>
</center>

<img style="position: relative;left:33%;float:left" src="https://miro.medium.com/proxy/1*93CVLqnQESmvfOhzvYUgQw.png" width=200><img style="position: relative;left:38%;float:left" src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Pandas_logo.svg/1200px-Pandas_logo.svg.png" width=200>
<br><br><br><br><br><br>
- Schnelle, flexible und aussagekräftige Datenstrukturen, welche die Arbeit mit relationalen Daten sowohl einfach als auch intuitiv macht.
- Fühlt sich an wie Excel

In [ ]:
import pandas as pd


tracks_overview_df = pd.io.json.json_normalize(tracks_overview['tracks'])
tracks_overview_df.head(10)

### Datensätze Filtern

In [ ]:
#tracks_overview_df[tracks_overview_df['sensor.properties.constructionYear'].eq(2014) & tracks_overview_df['sensor.properties.fuelType'].eq('diesel')]
tracks_overview_df[tracks_overview_df['sensor.properties.fuelType'].eq('diesel') & 
                   tracks_overview_df['sensor.properties.constructionYear'].ge(2014) & #isin([2014,2015,2016,2017,2018,2019,2020])
                   tracks_overview_df.length.ge(10)]

In [ ]:
import requests
car_id ="5e5c9ff877e02d42aa93004f"
track = requests.get("https://envirocar.org/api/stable/tracks/"+car_id).json()
track

### Datenstruktur anpassen (Flatten, Attribut Selection & Time)

### Flatten

In [ ]:
track_df = pd.io.json.json_normalize(track['features'])
track_df['car_id'] = car_id
track_df

### Attribut auswahl

In [ ]:
track_df[['x','y']] = pd.DataFrame(track_df['geometry.coordinates'].values.tolist())
track_df = track_df[['car_id','properties.time','x','y']].copy()
track_df.columns = ['id','time', 'x', 'y']
track_df.head(5)

##### convert_time() Helferfunktion

In [ ]:
#Helper function to convert Date String to Epoch Mili
from datetime import datetime
def convert_time(d):
    p = '%Y-%m-%dT%H:%M:%SZ'
    return int(datetime.strptime(d, p).timestamp()*1000)

### Feature Set erstellen inkl. Zeitfeld umwandeln und Feldtyp anpassen

In [ ]:
from arcgis.features import FeatureSet

for index, row in track_df.iterrows():
    track_df.loc[index,'time']=convert_time(track_df.loc[index,'time'])

track_fs = FeatureSet.from_dataframe(track_df)
for index, item in enumerate(track_fs.fields):
    if item['name'] == 'time':
        item['type'] = 'esriFieldTypeDate'

In [ ]:
track_fs

### Spatially Enabled Data Frame erstellen

<center>
<img src="https://demoportal.esri.de/bilder/esrikon2020/pandas_globe.png" width="400">
</center>

In [ ]:
track_sdf = pd.DataFrame.spatial.from_xy(track_fs.sdf,'x','y',sr=4326)
track_sdf.head(5)

### SDF Visualisieren

In [ ]:
m = GIS().map(str(track_sdf.iloc[0]['x'])+','+str(track_sdf.iloc[0]['y']))
m

In [ ]:
track_sdf.spatial.plot(map_widget=m)

### Feature Service publizieren

In [ ]:
track_layer = track_sdf.spatial.to_featurelayer('EnviroCar_Track_Export')
track_layer

##### Zeiteinstellung aktivieren

In [ ]:
timeInfoObject = {"timeInfo": {
    "startTimeField": "time",
    "endTimeField": None,
    "trackIdField": "id",
    "timeReference": {
        "timeZone": "UTC",
        "respectsDaylightSaving": False
    },
    "timeInterval": 0,
    "timeIntervalUnits": None,
    "exportOptions": {
        "useTime": False,
        "timeDataCumulative": False,
        "timeOffset": 0,
        "timeOffsetUnits": "esriTimeUnitsUnknown"
        },
    "hasLiveData": False,
    "timeExtent": [1580277349000,1580277753000]
}}

In [ ]:
track_layer.layers[0].manager.update_definition(timeInfoObject)

In [ ]:
track_layer.layers[0].url

### Features mit GeoAnalytics analisieren

In [ ]:
from arcgis.geoanalytics.summarize_data import reconstruct_tracks
analysis_result = reconstruct_tracks(track_layer.layers[0].url, 
                   track_fields='id', 
                   method = 'Geodesic',
                   summary_fields = [],
                   output_name = 'EnviroCar_Track_Export_Reconstructed_Tracks')

In [ ]:
analysis_result_sdf = pd.DataFrame.spatial.from_layer(analysis_result.layers[0])
analysis_result_sdf.spatial.plot(map_widget=m)
m